In [29]:
"""
================================================================================
PHASE 1: INDIVIDUAL DATASET ANALYSIS
================================================================================
Polymarket Bitcoin Bot Strategy Reverse-Engineering

This script analyzes each of the 4 datasets individually to understand:
1. Data quality, completeness, and temporal coverage
2. Basic statistical properties
3. Patterns that might hint at the bot's strategy

JUSTIFICATION FOR PHASE 1:
Before merging datasets, we need to understand each one individually:
- Verify data integrity and identify any gaps/anomalies
- Establish baseline statistics for comparison
- Identify obvious patterns that don't require cross-referencing
- Ensure timestamp formats are compatible for later merging

Author: Strategy Analysis Pipeline
Date: December 2024
================================================================================
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For visualization (optional - comment out if not needed)
try:
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    PLOTTING_AVAILABLE = True
except ImportError:
    PLOTTING_AVAILABLE = False
    print("matplotlib not available - will skip visualizations")

try:
    import seaborn as sns
    SEABORN_AVAILABLE = True
except ImportError:
    SEABORN_AVAILABLE = False


# ============================================================================
# CONFIGURATION - UPDATE THESE PATHS TO MATCH YOUR LOCAL SETUP
# ============================================================================

CONFIG = {
    'bitcoin_prices_path': 'polymarket_data/bitcoin_prices.parquet',
    'orderbook_path': 'polymarket_data/orderbook_snapshots.parquet',
    'positions_path': 'polymarket_data/positions_live.parquet',  # Update filename
    'trades_path': 'polymarket_data/trades_history.parquet',  # Update filename
    'output_dir': 'analysis_output/',
}

# Create output directory
import os
os.makedirs(CONFIG['output_dir'], exist_ok=True)


# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def print_section(title):
    """Print a formatted section header"""
    print("\n" + "="*80)
    print(f" {title}")
    print("="*80)

def print_subsection(title):
    """Print a formatted subsection header"""
    print(f"\n--- {title} ---")

def safe_describe(series, name=""):
    """Safely describe a series, handling edge cases"""
    print(f"\n{name}:")
    try:
        desc = series.describe()
        print(desc)
        return desc
    except Exception as e:
        print(f"  Error describing: {e}")
        return None

def convert_unix_timestamp(ts, unit='s'):
    """
    Convert unix timestamp to datetime.
    
    JUSTIFICATION:
    The datasets use different timestamp formats:
    - Some use seconds since epoch
    - Some use milliseconds or microseconds
    We need to normalize all to datetime for proper merging.
    """
    if ts > 1e12:  # Likely milliseconds
        return pd.to_datetime(ts, unit='ms')
    elif ts > 1e15:  # Likely microseconds
        return pd.to_datetime(ts, unit='us')
    else:  # Likely seconds
        return pd.to_datetime(ts, unit='s')


# ============================================================================
# 1A. BITCOIN PRICE ANALYSIS
# ============================================================================

def analyze_bitcoin_prices(filepath):
    """
    Comprehensive analysis of the Bitcoin price dataset.
    
    JUSTIFICATION:
    The BTC price is the fundamental input that likely drives the bot's decisions.
    Understanding price dynamics during the trading period tells us:
    - What market conditions the bot operates in
    - What price movements might trigger entries/exits
    - The baseline volatility the bot is trying to predict
    
    KEY QUESTIONS:
    - What's the tick frequency? (determines reaction time possible)
    - What's the typical price movement in a 15-min window?
    - Are there any data gaps that could affect analysis?
    """
    print_section("1A. BITCOIN PRICE ANALYSIS")
    
    try:
        df = pd.read_parquet(filepath)
        print(f"✓ Loaded {len(df):,} price records")
    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
        print("  Please update CONFIG['bitcoin_prices_path'] with the correct path")
        return None
    
    # Display schema
    print_subsection("Schema")
    print(df.dtypes)
    print(f"\nColumns: {list(df.columns)}")
    
    # Parse timestamps
    print_subsection("Timestamp Analysis")
    
    # Check if system_timestamp is already datetime
    if df['system_timestamp'].dtype == 'datetime64[ns]':
        df['datetime'] = df['system_timestamp']
    else:
        # Try to parse
        df['datetime'] = pd.to_datetime(df['system_timestamp'])
    
    print(f"Time range: {df['datetime'].min()} to {df['datetime'].max()}")
    print(f"Total duration: {df['datetime'].max() - df['datetime'].min()}")
    
    # Tick frequency analysis
    # JUSTIFICATION: Understanding tick frequency tells us the minimum reaction time
    # the bot can have - if ticks are every 100ms, bot can't react faster than that
    df_sorted = df.sort_values('datetime')
    df_sorted['time_delta'] = df_sorted['datetime'].diff().dt.total_seconds()
    
    print_subsection("Tick Frequency (time between price updates)")
    print(f"Mean: {df_sorted['time_delta'].mean():.4f} seconds")
    print(f"Median: {df_sorted['time_delta'].median():.4f} seconds")
    print(f"Min: {df_sorted['time_delta'].min():.4f} seconds")
    print(f"Max: {df_sorted['time_delta'].max():.4f} seconds")
    print(f"Std: {df_sorted['time_delta'].std():.4f} seconds")
    
    # Check for gaps (> 5 seconds between ticks)
    gaps = df_sorted[df_sorted['time_delta'] > 5]
    print(f"\nData gaps (>5 sec): {len(gaps)} occurrences")
    if len(gaps) > 0:
        print("Largest gaps:")
        print(gaps.nlargest(5, 'time_delta')[['datetime', 'time_delta']])
    
    # Price statistics
    print_subsection("Price Statistics")
    print(f"Mean price: ${df['price'].mean():,.2f}")
    print(f"Std dev: ${df['price'].std():,.2f}")
    print(f"Min: ${df['price'].min():,.2f}")
    print(f"Max: ${df['price'].max():,.2f}")
    print(f"Range: ${df['price'].max() - df['price'].min():,.2f}")
    print(f"Range %: {(df['price'].max() - df['price'].min()) / df['price'].mean() * 100:.4f}%")
    
    # Returns analysis
    # JUSTIFICATION: Returns (not raw prices) are what the bot likely uses for signals
    # A prediction market on "BTC up or down" cares about direction, not level
    print_subsection("Returns Analysis")
    df_sorted['return_1tick'] = df_sorted['price'].pct_change()
    df_sorted['return_abs'] = df_sorted['price'].diff()
    
    print(f"Mean 1-tick return: {df_sorted['return_1tick'].mean()*100:.6f}%")
    print(f"Std 1-tick return: {df_sorted['return_1tick'].std()*100:.6f}%")
    print(f"Max positive return: {df_sorted['return_1tick'].max()*100:.4f}%")
    print(f"Max negative return: {df_sorted['return_1tick'].min()*100:.4f}%")
    
    # Compute returns over different windows
    # JUSTIFICATION: Bot might use different lookback windows - we need to test several
    print_subsection("Multi-Window Returns (for later signal analysis)")
    
    # Approximate returns for different time windows
    # Note: This is approximate since ticks aren't perfectly spaced
    avg_tick_time = df_sorted['time_delta'].median()
    print(f"Average tick interval: {avg_tick_time:.3f}s")
    
    windows_seconds = [1, 5, 15, 30, 60, 300]  # 1s, 5s, 15s, 30s, 1min, 5min
    for window in windows_seconds:
        n_ticks = max(1, int(window / avg_tick_time))
        df_sorted[f'return_{window}s'] = df_sorted['price'].pct_change(n_ticks)
        std_return = df_sorted[f'return_{window}s'].std() * 100
        print(f"  {window:>3}s window (~{n_ticks} ticks): std = {std_return:.4f}%")
    
    # Volatility analysis
    # JUSTIFICATION: High volatility periods might be when the bot is most active
    # or conversely when it stays out
    print_subsection("Volatility Analysis")
    df_sorted['rolling_vol_60s'] = df_sorted['return_1tick'].rolling(
        window=int(60/avg_tick_time)
    ).std() * 100
    
    print(f"Mean 60s rolling volatility: {df_sorted['rolling_vol_60s'].mean():.6f}%")
    print(f"Max 60s rolling volatility: {df_sorted['rolling_vol_60s'].max():.6f}%")
    
    # Direction analysis - crucial for up/down market
    # JUSTIFICATION: This is literally what the prediction market is about
    print_subsection("Direction Analysis (Up vs Down moves)")
    up_moves = (df_sorted['return_1tick'] > 0).sum()
    down_moves = (df_sorted['return_1tick'] < 0).sum()
    flat = (df_sorted['return_1tick'] == 0).sum()
    
    print(f"Up ticks: {up_moves:,} ({up_moves/len(df_sorted)*100:.1f}%)")
    print(f"Down ticks: {down_moves:,} ({down_moves/len(df_sorted)*100:.1f}%)")
    print(f"Flat ticks: {flat:,} ({flat/len(df_sorted)*100:.1f}%)")
    
    # Save processed data
    output_path = CONFIG['output_dir'] + 'btc_prices_processed.parquet'
    df_sorted.to_parquet(output_path)
    print(f"\n✓ Saved processed BTC data to: {output_path}")
    
    return df_sorted


# ============================================================================
# 1B. TRADE ANALYSIS
# ============================================================================

def analyze_trades(filepath):
    """
    Comprehensive analysis of the bot's executed trades.
    
    JUSTIFICATION:
    This is THE GOLDMINE. Every trade is a decision the bot made.
    By analyzing trade patterns, we can understand:
    - When does the bot enter/exit?
    - What sizes does it use?
    - Does it prefer UP or DOWN outcomes?
    - What prices does it pay (market vs limit)?
    
    KEY QUESTIONS:
    - What's the distribution of BUY vs SELL?
    - What's the distribution of UP vs DOWN outcomes?
    - Are trade sizes fixed or variable?
    - What's the timing pattern within 15-min windows?
    """
    print_section("1B. TRADE ANALYSIS (THE GOLDMINE)")
    
    try:
        df = pd.read_parquet(filepath)
        print(f"✓ Loaded {len(df):,} trades")
    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
        print("  Please update CONFIG['trades_path'] with the correct path")
        return None
    
    # Display schema
    print_subsection("Schema")
    print(df.dtypes)
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nFirst 3 rows:")
    print(df.head(3))
    
    # Parse timestamps
    print_subsection("Timestamp Analysis")
    
    # Check if timestamp is already datetime, otherwise convert
    if 'timestamp' in df.columns and pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df['datetime'] = df['timestamp']
    elif 'unix_timestamp' in df.columns:
        df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='s')
    elif 'timestamp' in df.columns:
        df['datetime'] = pd.to_datetime(df['timestamp'])
    
    print(f"Time range: {df['datetime'].min()} to {df['datetime'].max()}")
    print(f"Total duration: {df['datetime'].max() - df['datetime'].min()}")
    
    # Trade count and frequency
    print_subsection("Trade Frequency")
    total_seconds = (df['datetime'].max() - df['datetime'].min()).total_seconds()
    trades_per_minute = len(df) / (total_seconds / 60) if total_seconds > 0 else 0
    print(f"Total trades: {len(df):,}")
    print(f"Trades per minute: {trades_per_minute:.2f}")
    
    # Inter-trade time analysis
    df_sorted = df.sort_values('datetime')
    df_sorted['time_since_last_trade'] = df_sorted['datetime'].diff().dt.total_seconds()
    
    print(f"\nTime between trades:")
    print(f"  Mean: {df_sorted['time_since_last_trade'].mean():.2f} seconds")
    print(f"  Median: {df_sorted['time_since_last_trade'].median():.2f} seconds")
    print(f"  Min: {df_sorted['time_since_last_trade'].min():.2f} seconds")
    print(f"  Max: {df_sorted['time_since_last_trade'].max():.2f} seconds")
    
    # Side analysis (BUY vs SELL)
    # JUSTIFICATION: Understanding when bot buys vs sells reveals entry/exit logic
    print_subsection("Side Analysis (BUY vs SELL)")
    side_counts = df['side'].value_counts()
    print(side_counts)
    print(f"\nBuy/Sell Ratio: {side_counts.get('BUY', 0) / max(side_counts.get('SELL', 1), 1):.2f}")
    
    # Outcome analysis (UP vs DOWN)
    # JUSTIFICATION: Does bot favor one direction? Or trade both equally?
    print_subsection("Outcome Analysis (UP vs DOWN)")
    outcome_counts = df['outcome'].value_counts()
    print(outcome_counts)
    
    # Combined side + outcome analysis
    # JUSTIFICATION: The 4 combinations reveal the full strategy:
    # BUY UP = betting BTC goes up
    # SELL UP = closing a BTC up bet OR shorting
    # BUY DOWN = betting BTC goes down
    # SELL DOWN = closing a BTC down bet OR shorting
    print_subsection("Combined Side + Outcome Analysis")
    df['action'] = df['side'] + '_' + df['outcome']
    df_sorted['action'] = df_sorted['side'] + '_' + df_sorted['outcome']  # Add to sorted df too
    action_counts = df['action'].value_counts()
    print(action_counts)
    print(f"\nAction distribution (%):")
    print((action_counts / len(df) * 100).round(2))
    
    # Trade size analysis
    # JUSTIFICATION: Fixed sizes suggest systematic strategy; variable sizes suggest scaling
    print_subsection("Trade Size Analysis")
    print(f"Size statistics:")
    print(f"  Mean: {df['size'].mean():.4f}")
    print(f"  Median: {df['size'].median():.4f}")
    print(f"  Std: {df['size'].std():.4f}")
    print(f"  Min: {df['size'].min():.4f}")
    print(f"  Max: {df['size'].max():.4f}")
    
    # Check if sizes are fixed
    unique_sizes = df['size'].nunique()
    print(f"\nUnique trade sizes: {unique_sizes}")
    if unique_sizes <= 10:
        print("Size distribution:")
        print(df['size'].value_counts().head(10))
    
    # Size by action
    print("\nMean size by action:")
    print(df.groupby('action')['size'].mean())
    
    # Trade price analysis
    # JUSTIFICATION: Prices reveal if bot is paying spread (market orders) 
    # or getting filled at limit prices
    print_subsection("Trade Price Analysis")
    print(f"Price statistics:")
    print(f"  Mean: {df['price'].mean():.4f}")
    print(f"  Median: {df['price'].median():.4f}")
    print(f"  Min: {df['price'].min():.4f}")
    print(f"  Max: {df['price'].max():.4f}")
    
    # Price by outcome
    print("\nMean price by outcome:")
    print(df.groupby('outcome')['price'].agg(['mean', 'min', 'max']))
    
    # Price by action
    print("\nMean price by action:")
    print(df.groupby('action')['price'].mean())
    
    # Trade value analysis
    # JUSTIFICATION: Total capital deployed per trade
    print_subsection("Trade Value Analysis (price * size)")
    if 'value' in df.columns:
        print(f"Value statistics:")
        print(f"  Mean: ${df['value'].mean():.4f}")
        print(f"  Total: ${df['value'].sum():.2f}")
    else:
        df['value'] = df['price'] * df['size']
        print(f"Computed value (price * size):")
        print(f"  Mean: ${df['value'].mean():.4f}")
        print(f"  Total: ${df['value'].sum():.2f}")
    
    # Timing within market window analysis
    # JUSTIFICATION: 15-minute markets have a specific lifecycle
    # Bot might trade more at open, close, or specific times
    print_subsection("Timing Within 15-Minute Windows")
    
    # Extract the market slug to understand which 15-min window
    if 'slug' in df.columns or 'market_slug' in df.columns:
        slug_col = 'slug' if 'slug' in df.columns else 'market_slug'
        print(f"Markets traded:")
        print(df[slug_col].value_counts())
    
    # Calculate seconds into each 15-minute window
    # The market slug contains the start timestamp
    # Format: btc-updown-15m-{unix_timestamp}
    if 'slug' in df.columns:
        try:
            # Extract market start time from slug
            df['market_start_unix'] = df['slug'].str.extract(r'(\d+)$').astype(float)
            df['market_start'] = pd.to_datetime(df['market_start_unix'], unit='s')
            df['seconds_into_window'] = (df['datetime'] - df['market_start']).dt.total_seconds()
            
            print(f"\nSeconds into 15-min window when trading:")
            print(f"  Mean: {df['seconds_into_window'].mean():.1f}s")
            print(f"  Median: {df['seconds_into_window'].median():.1f}s")
            print(f"  Min: {df['seconds_into_window'].min():.1f}s")
            print(f"  Max: {df['seconds_into_window'].max():.1f}s")
            
            # Bin into early/mid/late
            df['window_phase'] = pd.cut(
                df['seconds_into_window'], 
                bins=[0, 300, 600, 900],  # 0-5min, 5-10min, 10-15min
                labels=['Early (0-5min)', 'Mid (5-10min)', 'Late (10-15min)']
            )
            print("\nTrades by window phase:")
            print(df['window_phase'].value_counts())
        except Exception as e:
            print(f"Could not parse market timing: {e}")
    
    # Sequence analysis
    # JUSTIFICATION: Understanding trade sequences reveals position building/closing patterns
    print_subsection("Trade Sequence Analysis")
    
    # Look at consecutive trades
    df_sorted['prev_action'] = df_sorted['action'].shift(1)
    df_sorted['action_sequence'] = df_sorted['prev_action'] + ' → ' + df_sorted['action']
    
    print("Most common action sequences:")
    print(df_sorted['action_sequence'].value_counts().head(10))
    
    # Check for rapid sequences (trades within 5 seconds)
    rapid_trades = df_sorted[df_sorted['time_since_last_trade'] < 5]
    print(f"\nRapid trades (<5 sec apart): {len(rapid_trades)} ({len(rapid_trades)/len(df)*100:.1f}%)")
    if len(rapid_trades) > 0:
        print("Rapid trade sequences:")
        print(rapid_trades['action_sequence'].value_counts().head(5))
    
    # Save processed data
    output_path = CONFIG['output_dir'] + 'trades_processed.parquet'
    df_sorted.to_parquet(output_path)
    print(f"\n✓ Saved processed trades data to: {output_path}")
    
    return df_sorted


# ============================================================================
# 1C. POSITION ANALYSIS
# ============================================================================

def analyze_positions(filepath):
    """
    Comprehensive analysis of position snapshots.
    
    JUSTIFICATION:
    Position data shows the bot's exposure over time:
    - How positions evolve from entry to exit
    - PnL characteristics (wins vs losses)
    - Position sizing patterns
    - Risk management behavior
    
    KEY QUESTIONS:
    - What's the typical position size?
    - How long are positions held?
    - What's the win rate and average win/loss?
    - Does the bot cut losses or let them run?
    """
    print_section("1C. POSITION ANALYSIS")
    
    try:
        df = pd.read_parquet(filepath)
        print(f"✓ Loaded {len(df):,} position snapshots")
    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
        print("  Please update CONFIG['positions_path'] with the correct path")
        return None
    
    # Display schema
    print_subsection("Schema")
    print(df.dtypes)
    print(f"\nColumns: {list(df.columns)}")
    
    # Basic stats
    print_subsection("Basic Statistics")
    print(f"Total snapshots: {len(df):,}")
    
    # Unique positions
    if 'position_id' in df.columns:
        print(f"Unique position IDs: {df['position_id'].nunique()}")
    
    # Outcome distribution
    print_subsection("Outcome Distribution (UP vs DOWN positions)")
    if 'outcome' in df.columns:
        print(df['outcome'].value_counts())
    
    # Shares analysis
    # JUSTIFICATION: Position sizes reveal risk appetite and scaling behavior
    print_subsection("Shares (Position Size) Analysis")
    print(f"Shares statistics:")
    print(f"  Mean: {df['shares'].mean():.2f}")
    print(f"  Median: {df['shares'].median():.2f}")
    print(f"  Std: {df['shares'].std():.2f}")
    print(f"  Max: {df['shares'].max():.2f}")
    
    print("\nShares by outcome:")
    print(df.groupby('outcome')['shares'].describe())
    
    # Price analysis
    # JUSTIFICATION: avg_price vs current_price shows position profitability
    print_subsection("Price Analysis")
    print(f"Average entry price (avg_price):")
    print(f"  Mean: {df['avg_price'].mean():.4f}")
    print(f"  Range: {df['avg_price'].min():.4f} to {df['avg_price'].max():.4f}")
    
    print(f"\nCurrent price:")
    print(f"  Mean: {df['current_price'].mean():.4f}")
    print(f"  Range: {df['current_price'].min():.4f} to {df['current_price'].max():.4f}")
    
    # PnL Analysis - THIS IS CRITICAL
    # JUSTIFICATION: PnL distribution reveals strategy effectiveness
    print_subsection("PnL Analysis (CRITICAL)")
    
    # Cash PnL
    print(f"Cash PnL statistics:")
    print(f"  Mean: ${df['cash_pnl'].mean():.2f}")
    print(f"  Median: ${df['cash_pnl'].median():.2f}")
    print(f"  Std: ${df['cash_pnl'].std():.2f}")
    print(f"  Min: ${df['cash_pnl'].min():.2f}")
    print(f"  Max: ${df['cash_pnl'].max():.2f}")
    
    # Percent PnL
    print(f"\nPercent PnL statistics:")
    print(f"  Mean: {df['percent_pnl'].mean():.2f}%")
    print(f"  Median: {df['percent_pnl'].median():.2f}%")
    print(f"  Min: {df['percent_pnl'].min():.2f}%")
    print(f"  Max: {df['percent_pnl'].max():.2f}%")
    
    # Win/Loss analysis
    winning = df[df['cash_pnl'] > 0]
    losing = df[df['cash_pnl'] < 0]
    breakeven = df[df['cash_pnl'] == 0]
    
    print(f"\nWin/Loss breakdown (by snapshot):")
    print(f"  Winning: {len(winning):,} ({len(winning)/len(df)*100:.1f}%)")
    print(f"  Losing: {len(losing):,} ({len(losing)/len(df)*100:.1f}%)")
    print(f"  Breakeven: {len(breakeven):,} ({len(breakeven)/len(df)*100:.1f}%)")
    
    if len(winning) > 0:
        print(f"\nAverage winning PnL: ${winning['cash_pnl'].mean():.2f}")
    if len(losing) > 0:
        print(f"Average losing PnL: ${losing['cash_pnl'].mean():.2f}")
    
    # PnL by outcome
    print("\nPnL by outcome (UP vs DOWN positions):")
    print(df.groupby('outcome')['cash_pnl'].describe())
    
    # Realized vs Unrealized PnL
    # JUSTIFICATION: Shows if bot is locking in profits or letting them ride
    print_subsection("Realized vs Unrealized PnL")
    if 'realized_pnl' in df.columns:
        print(f"Realized PnL:")
        print(f"  Mean: ${df['realized_pnl'].mean():.2f}")
        print(f"  Total: ${df['realized_pnl'].sum():.2f}")
        
        print(f"\nPercent Realized PnL:")
        print(f"  Mean: {df['percent_realized_pnl'].mean():.2f}%")
    
    # Value analysis
    print_subsection("Position Value Analysis")
    print(f"Initial value:")
    print(f"  Mean: ${df['initial_value'].mean():.2f}")
    print(f"  Total: ${df['initial_value'].sum():.2f}")
    
    print(f"\nCurrent value:")
    print(f"  Mean: ${df['current_value'].mean():.2f}")
    print(f"  Total: ${df['current_value'].sum():.2f}")
    
    # Total bought
    print(f"\nTotal bought:")
    print(f"  Mean: ${df['total_bought'].mean():.2f}")
    print(f"  Max: ${df['total_bought'].max():.2f}")
    
    # Redeemable status
    # JUSTIFICATION: Shows if positions are being closed at market resolution
    print_subsection("Redeemable Status")
    if 'redeemable' in df.columns:
        print(df['redeemable'].value_counts())
    
    # End date analysis
    print_subsection("Market End Date Analysis")
    if 'end_date' in df.columns:
        print(f"End dates:")
        print(df['end_date'].value_counts())
    
    # Save processed data
    output_path = CONFIG['output_dir'] + 'positions_processed.parquet'
    df.to_parquet(output_path)
    print(f"\n✓ Saved processed positions data to: {output_path}")
    
    return df


# ============================================================================
# 1D. ORDERBOOK ANALYSIS
# ============================================================================

def analyze_orderbook(filepath):
    """
    Comprehensive analysis of orderbook snapshots.
    
    JUSTIFICATION:
    The orderbook reveals market microstructure:
    - What prices are available when bot trades
    - Spread conditions (cost of trading)
    - Market-implied probabilities
    - Liquidity conditions
    
    KEY QUESTIONS:
    - What's the typical spread?
    - How do UP and DOWN prices relate?
    - Are there arbitrage opportunities?
    - When is liquidity best/worst?
    """
    print_section("1D. ORDERBOOK ANALYSIS")
    
    try:
        df = pd.read_parquet(filepath)
        print(f"✓ Loaded {len(df):,} orderbook snapshots")
    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
        print("  Please update CONFIG['orderbook_path'] with the correct path")
        return None
    
    # Display schema
    print_subsection("Schema")
    print(df.dtypes)
    print(f"\nColumns: {list(df.columns)}")
    
    # Parse timestamps
    print_subsection("Timestamp Analysis")
    if 'timestamp' in df.columns:
        df['datetime'] = pd.to_datetime(df['timestamp'])
    elif 'unixtime' in df.columns:
        df['datetime'] = pd.to_datetime(df['unixtime'], unit='s')
    
    print(f"Time range: {df['datetime'].min()} to {df['datetime'].max()}")
    print(f"Total duration: {df['datetime'].max() - df['datetime'].min()}")
    
    # Snapshot frequency
    df_sorted = df.sort_values('datetime')
    df_sorted['time_delta'] = df_sorted['datetime'].diff().dt.total_seconds()
    print(f"\nSnapshot frequency:")
    print(f"  Mean: {df_sorted['time_delta'].mean():.2f} seconds")
    print(f"  Median: {df_sorted['time_delta'].median():.2f} seconds")
    
    # Market slug analysis
    print_subsection("Market Coverage")
    if 'market_slug' in df.columns:
        print(df['market_slug'].value_counts())
    
    # UP outcome analysis
    # JUSTIFICATION: UP prices tell us market's view on BTC going up
    print_subsection("UP Outcome Orderbook")
    print(f"UP Best Bid:")
    print(f"  Mean: {df['up_best_bid'].mean():.4f}")
    print(f"  Min: {df['up_best_bid'].min():.4f}")
    print(f"  Max: {df['up_best_bid'].max():.4f}")
    
    print(f"\nUP Best Ask:")
    print(f"  Mean: {df['up_best_ask'].mean():.4f}")
    print(f"  Min: {df['up_best_ask'].min():.4f}")
    print(f"  Max: {df['up_best_ask'].max():.4f}")
    
    print(f"\nUP Spread:")
    print(f"  Mean: {df['up_spread'].mean():.4f}")
    print(f"  Min: {df['up_spread'].min():.4f}")
    print(f"  Max: {df['up_spread'].max():.4f}")
    
    # DOWN outcome analysis
    print_subsection("DOWN Outcome Orderbook")
    print(f"DOWN Best Bid:")
    print(f"  Mean: {df['down_best_bid'].mean():.4f}")
    print(f"  Min: {df['down_best_bid'].min():.4f}")
    print(f"  Max: {df['down_best_bid'].max():.4f}")
    
    print(f"\nDOWN Best Ask:")
    print(f"  Mean: {df['down_best_ask'].mean():.4f}")
    print(f"  Min: {df['down_best_ask'].min():.4f}")
    print(f"  Max: {df['down_best_ask'].max():.4f}")
    
    print(f"\nDOWN Spread:")
    print(f"  Mean: {df['down_spread'].mean():.4f}")
    print(f"  Min: {df['down_spread'].min():.4f}")
    print(f"  Max: {df['down_spread'].max():.4f}")
    
    # Midpoint prices (implied probabilities)
    # JUSTIFICATION: Midpoint = market's implied probability of that outcome
    print_subsection("Implied Probabilities (Midpoint Prices)")
    df['up_midpoint'] = (df['up_best_bid'] + df['up_best_ask']) / 2
    df['down_midpoint'] = (df['down_best_bid'] + df['down_best_ask']) / 2
    
    print(f"UP implied probability (midpoint):")
    print(f"  Mean: {df['up_midpoint'].mean():.4f} ({df['up_midpoint'].mean()*100:.1f}%)")
    print(f"  Range: {df['up_midpoint'].min():.4f} to {df['up_midpoint'].max():.4f}")
    
    print(f"\nDOWN implied probability (midpoint):")
    print(f"  Mean: {df['down_midpoint'].mean():.4f} ({df['down_midpoint'].mean()*100:.1f}%)")
    print(f"  Range: {df['down_midpoint'].min():.4f} to {df['down_midpoint'].max():.4f}")
    
    # Probability sum analysis
    # JUSTIFICATION: UP + DOWN should = 1 in efficient market
    # Deviation = overround (house edge) or arbitrage opportunity
    print_subsection("Market Efficiency Analysis")
    df['prob_sum_mid'] = df['up_midpoint'] + df['down_midpoint']
    df['prob_sum_bid'] = df['up_best_bid'] + df['down_best_bid']  # Can buy both for this
    df['prob_sum_ask'] = df['up_best_ask'] + df['down_best_ask']  # Can sell both for this
    
    print(f"Sum of midpoint probabilities (should be ~1.0):")
    print(f"  Mean: {df['prob_sum_mid'].mean():.4f}")
    print(f"  Min: {df['prob_sum_mid'].min():.4f}")
    print(f"  Max: {df['prob_sum_mid'].max():.4f}")
    
    # If you can buy both UP and DOWN for < $1, that's an arb
    print(f"\nSum of best asks (cost to buy both outcomes):")
    print(f"  Mean: {df['prob_sum_ask'].mean():.4f}")
    print(f"  Min: {df['prob_sum_ask'].min():.4f}")
    
    # If you can sell both UP and DOWN for > $1, that's an arb
    print(f"\nSum of best bids (revenue from selling both outcomes):")
    print(f"  Mean: {df['prob_sum_bid'].mean():.4f}")
    print(f"  Max: {df['prob_sum_bid'].max():.4f}")
    
    # Check for arbitrage opportunities
    arb_buy = df[df['prob_sum_ask'] < 1.0]
    arb_sell = df[df['prob_sum_bid'] > 1.0]
    print(f"\nArbitrage opportunities (buy both < $1): {len(arb_buy)} snapshots")
    print(f"Arbitrage opportunities (sell both > $1): {len(arb_sell)} snapshots")
    
    # Depth analysis
    # JUSTIFICATION: Depth = how much can be traded at current prices
    print_subsection("Orderbook Depth Analysis")
    if 'up_bid_depth_5' in df.columns:
        print(f"UP Bid Depth (5 levels):")
        print(f"  Mean: ${df['up_bid_depth_5'].mean():.2f}")
        print(f"  Max: ${df['up_bid_depth_5'].max():.2f}")
        
        print(f"\nUP Ask Depth (5 levels):")
        print(f"  Mean: ${df['up_ask_depth_5'].mean():.2f}")
        print(f"  Max: ${df['up_ask_depth_5'].max():.2f}")
    
    # Implied sum analysis
    # JUSTIFICATION: These appear to be the total implied probabilities from orderbook
    print_subsection("Implied Sum (Bid/Ask)")
    if 'implied_sum_bid' in df.columns:
        print(f"Implied Sum Bid: {df['implied_sum_bid'].mean():.4f}")
        print(f"Implied Sum Ask: {df['implied_sum_ask'].mean():.4f}")
    
    # Spread comparison
    # JUSTIFICATION: If one outcome has tighter spread, might be more liquid/efficient
    print_subsection("UP vs DOWN Spread Comparison")
    df['spread_diff'] = df['up_spread'] - df['down_spread']
    print(f"UP spread - DOWN spread:")
    print(f"  Mean: {df['spread_diff'].mean():.4f}")
    print(f"  Positive (UP wider): {(df['spread_diff'] > 0).sum()} snapshots")
    print(f"  Negative (DOWN wider): {(df['spread_diff'] < 0).sum()} snapshots")
    
    # Save processed data
    output_path = CONFIG['output_dir'] + 'orderbook_processed.parquet'
    df.to_parquet(output_path)
    print(f"\n✓ Saved processed orderbook data to: {output_path}")
    
    return df


# ============================================================================
# SUMMARY REPORT
# ============================================================================

def generate_summary_report(btc_df, trades_df, positions_df, orderbook_df):
    """Generate a summary report of Phase 1 findings"""
    print_section("PHASE 1 SUMMARY REPORT")
    
    print("\n📊 DATA OVERVIEW")
    print("-" * 40)
    
    if btc_df is not None:
        print(f"Bitcoin Prices: {len(btc_df):,} records")
    if trades_df is not None:
        print(f"Trades: {len(trades_df):,} records")
    if positions_df is not None:
        print(f"Position Snapshots: {len(positions_df):,} records")
    if orderbook_df is not None:
        print(f"Orderbook Snapshots: {len(orderbook_df):,} records")
    
    print("\n🔍 KEY FINDINGS")
    print("-" * 40)
    
    if trades_df is not None:
        print("\nTRADING BEHAVIOR:")
        if 'action' in trades_df.columns:
            action_pcts = trades_df['action'].value_counts(normalize=True) * 100
            for action, pct in action_pcts.items():
                print(f"  {action}: {pct:.1f}%")
        
        print(f"\n  Trade size mode: {trades_df['size'].mode().values[0] if len(trades_df['size'].mode()) > 0 else 'N/A'}")
        
        if 'seconds_into_window' in trades_df.columns:
            print(f"  Avg entry time: {trades_df['seconds_into_window'].mean():.0f}s into window")
    
    if positions_df is not None:
        print("\nPOSITION PERFORMANCE:")
        print(f"  Mean PnL: ${positions_df['cash_pnl'].mean():.2f}")
        win_rate = (positions_df['cash_pnl'] > 0).mean() * 100
        print(f"  Win rate (by snapshot): {win_rate:.1f}%")
    
    if orderbook_df is not None:
        print("\nMARKET CONDITIONS:")
        print(f"  Avg UP spread: {orderbook_df['up_spread'].mean():.4f}")
        print(f"  Avg DOWN spread: {orderbook_df['down_spread'].mean():.4f}")
        if 'up_midpoint' in orderbook_df.columns:
            print(f"  Avg UP implied prob: {orderbook_df['up_midpoint'].mean()*100:.1f}%")
    
    print("\n📋 NEXT STEPS (Phase 2)")
    print("-" * 40)
    print("1. Merge trades with BTC prices to find entry triggers")
    print("2. Merge trades with orderbook to analyze execution")
    print("3. Build full state reconstruction")
    print("4. Run decision tree analysis to extract rules")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Run the complete Phase 1 analysis"""
    print("="*80)
    print(" POLYMARKET BOT STRATEGY ANALYSIS - PHASE 1")
    print(" Individual Dataset Analysis")
    print("="*80)
    print(f"\nStarted at: {datetime.now()}")
    print(f"Output directory: {CONFIG['output_dir']}")
    
    # Run each analysis
    btc_df = analyze_bitcoin_prices(CONFIG['bitcoin_prices_path'])
    trades_df = analyze_trades(CONFIG['trades_path'])
    positions_df = analyze_positions(CONFIG['positions_path'])
    orderbook_df = analyze_orderbook(CONFIG['orderbook_path'])
    
    # Generate summary
    generate_summary_report(btc_df, trades_df, positions_df, orderbook_df)
    
    print(f"\n✓ Phase 1 analysis complete at: {datetime.now()}")
    print(f"✓ Processed data saved to: {CONFIG['output_dir']}")
    print("\nRun Phase 2 for merged analysis and strategy identification.")
    
    return {
        'btc': btc_df,
        'trades': trades_df,
        'positions': positions_df,
        'orderbook': orderbook_df
    }


if __name__ == "__main__":
    results = main()


 POLYMARKET BOT STRATEGY ANALYSIS - PHASE 1
 Individual Dataset Analysis

Started at: 2025-12-22 22:02:17.146059
Output directory: analysis_output/

 1A. BITCOIN PRICE ANALYSIS
✓ Loaded 1,782 price records

--- Schema ---
system_timestamp    datetime64[ns]
server_timestamp             int64
symbol                      object
price                      float64
dtype: object

Columns: ['system_timestamp', 'server_timestamp', 'symbol', 'price']

--- Timestamp Analysis ---
Time range: 2025-12-21 16:14:13.142747 to 2025-12-21 16:44:29.922066
Total duration: 0 days 00:30:16.779319

--- Tick Frequency (time between price updates) ---
Mean: 1.0201 seconds
Median: 0.8533 seconds
Min: 0.3258 seconds
Max: 3.4176 seconds
Std: 0.3834 seconds

Data gaps (>5 sec): 0 occurrences

--- Price Statistics ---
Mean price: $89,045.85
Std dev: $192.85
Min: $88,762.78
Max: $89,586.20
Range: $823.41
Range %: 0.9247%

--- Returns Analysis ---
Mean 1-tick return: -0.000203%
Std 1-tick return: 0.009143%
Max positi

In [34]:
"""
================================================================================
PHASE 1 VISUALIZATIONS
================================================================================
Generates charts for visual analysis of individual datasets.

Run this AFTER phase1_individual_analysis.py to create visualizations.

JUSTIFICATION:
Visual analysis helps identify patterns that statistics alone might miss:
- Distributions reveal outliers and skewness
- Time series show temporal patterns
- Histograms reveal clustering in decision variables
================================================================================
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import os

# Try to import seaborn for better styling
try:
    import seaborn as sns
    sns.set_style("whitegrid")
    SEABORN_AVAILABLE = True
except ImportError:
    SEABORN_AVAILABLE = False

# Configuration - same as main script
CONFIG = {
    'btc_processed': 'analysis_output/btc_prices_processed.parquet',
    'trades_processed': 'analysis_output/trades_processed.parquet',
    'positions_processed': 'analysis_output/positions_processed.parquet',
    'orderbook_processed': 'analysis_output/orderbook_processed.parquet',
    'output_dir': 'analysis_output/charts/',
}

os.makedirs(CONFIG['output_dir'], exist_ok=True)


def plot_btc_analysis(df):
    """
    Create visualizations for Bitcoin price data.
    
    JUSTIFICATION:
    - Price chart shows overall market context
    - Return distribution reveals if normal or fat-tailed
    - Volatility chart shows when market is calm vs turbulent
    """
    if df is None:
        print("No BTC data to plot")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Bitcoin Price Analysis', fontsize=14, fontweight='bold')
    
    # 1. Price over time
    ax1 = axes[0, 0]
    ax1.plot(df['datetime'], df['price'], linewidth=0.5, alpha=0.8)
    ax1.set_title('BTC Price Over Time')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Price (USD)')
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. Return distribution
    ax2 = axes[0, 1]
    returns = df['return_1tick'].dropna()
    ax2.hist(returns * 100, bins=100, edgecolor='black', alpha=0.7)
    ax2.axvline(x=0, color='red', linestyle='--', label='Zero')
    ax2.set_title('1-Tick Return Distribution')
    ax2.set_xlabel('Return (%)')
    ax2.set_ylabel('Frequency')
    ax2.set_xlim(-0.1, 0.1)  # Focus on typical range
    
    # 3. Rolling volatility
    ax3 = axes[1, 0]
    if 'rolling_vol_60s' in df.columns:
        ax3.plot(df['datetime'], df['rolling_vol_60s'], linewidth=0.5, alpha=0.8)
        ax3.set_title('60-Second Rolling Volatility')
        ax3.set_xlabel('Time')
        ax3.set_ylabel('Volatility (%)')
        ax3.tick_params(axis='x', rotation=45)
    
    # 4. Tick interval distribution
    ax4 = axes[1, 1]
    if 'time_delta' in df.columns:
        valid_deltas = df['time_delta'].dropna()
        valid_deltas = valid_deltas[valid_deltas < 10]  # Focus on typical range
        ax4.hist(valid_deltas, bins=50, edgecolor='black', alpha=0.7)
        ax4.set_title('Time Between Ticks Distribution')
        ax4.set_xlabel('Seconds')
        ax4.set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.savefig(CONFIG['output_dir'] + 'btc_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved: {CONFIG['output_dir']}btc_analysis.png")


def plot_trade_analysis(df):
    """
    Create visualizations for trade data.
    
    JUSTIFICATION:
    - Action distribution shows strategy bias
    - Size distribution reveals position sizing logic
    - Timing histogram shows when bot is active within windows
    - Sequence chart shows trade patterns
    """
    if df is None:
        print("No trade data to plot")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Trade Analysis', fontsize=14, fontweight='bold')
    
    # 1. Action distribution (pie/bar)
    ax1 = axes[0, 0]
    if 'action' in df.columns:
        action_counts = df['action'].value_counts()
        colors = ['#2ecc71', '#e74c3c', '#3498db', '#f39c12'][:len(action_counts)]
        bars = ax1.bar(action_counts.index, action_counts.values, color=colors, edgecolor='black')
        ax1.set_title('Trade Actions Distribution')
        ax1.set_ylabel('Count')
        ax1.tick_params(axis='x', rotation=45)
        
        # Add percentage labels
        total = action_counts.sum()
        for bar, count in zip(bars, action_counts.values):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                    f'{count/total*100:.1f}%', ha='center', va='bottom', fontsize=10)
    
    # 2. Trade size distribution
    ax2 = axes[0, 1]
    ax2.hist(df['size'], bins=30, edgecolor='black', alpha=0.7)
    ax2.axvline(x=df['size'].mean(), color='red', linestyle='--', label=f'Mean: {df["size"].mean():.2f}')
    ax2.axvline(x=df['size'].median(), color='orange', linestyle='--', label=f'Median: {df["size"].median():.2f}')
    ax2.set_title('Trade Size Distribution')
    ax2.set_xlabel('Size')
    ax2.set_ylabel('Frequency')
    ax2.legend()
    
    # 3. Trade timing within 15-min window
    ax3 = axes[1, 0]
    if 'seconds_into_window' in df.columns:
        # Histogram of when trades occur
        ax3.hist(df['seconds_into_window'], bins=30, edgecolor='black', alpha=0.7, range=(0, 900))
        ax3.axvline(x=300, color='gray', linestyle='--', alpha=0.5, label='5 min')
        ax3.axvline(x=600, color='gray', linestyle='--', alpha=0.5, label='10 min')
        ax3.set_title('Trade Timing Within 15-Min Window')
        ax3.set_xlabel('Seconds into Window')
        ax3.set_ylabel('Number of Trades')
        ax3.set_xlim(0, 900)
        ax3.legend()
    
    # 4. Inter-trade time distribution
    ax4 = axes[1, 1]
    if 'time_since_last_trade' in df.columns:
        valid_times = df['time_since_last_trade'].dropna()
        valid_times = valid_times[valid_times < 120]  # Focus on <2 min
        ax4.hist(valid_times, bins=50, edgecolor='black', alpha=0.7)
        ax4.set_title('Time Between Consecutive Trades')
        ax4.set_xlabel('Seconds')
        ax4.set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.savefig(CONFIG['output_dir'] + 'trade_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved: {CONFIG['output_dir']}trade_analysis.png")


def plot_trade_price_analysis(df):
    """
    Additional trade analysis focusing on prices.
    
    JUSTIFICATION:
    - Price by outcome reveals what prices bot pays for UP vs DOWN
    - Trade price over time shows if strategy is consistent
    """
    if df is None:
        print("No trade data to plot")
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle('Trade Price Analysis', fontsize=14, fontweight='bold')
    
    # 1. Trade price by action
    ax1 = axes[0]
    if 'action' in df.columns:
        # Box plot of prices by action
        actions = df['action'].unique()
        data_to_plot = [df[df['action'] == a]['price'].values for a in actions]
        bp = ax1.boxplot(data_to_plot, labels=actions, patch_artist=True)
        colors = ['#2ecc71', '#e74c3c', '#3498db', '#f39c12'][:len(actions)]
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        ax1.set_title('Trade Price by Action')
        ax1.set_ylabel('Price')
        ax1.tick_params(axis='x', rotation=45)
    
    # 2. Trade price over time
    ax2 = axes[1]
    if 'datetime' in df.columns:
        for outcome in df['outcome'].unique():
            subset = df[df['outcome'] == outcome]
            ax2.scatter(subset['datetime'], subset['price'], 
                       alpha=0.5, s=20, label=outcome)
        ax2.set_title('Trade Prices Over Time')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Price')
        ax2.legend()
        ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(CONFIG['output_dir'] + 'trade_price_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved: {CONFIG['output_dir']}trade_price_analysis.png")


def plot_position_analysis(df):
    """
    Create visualizations for position data.
    
    JUSTIFICATION:
    - PnL distribution shows strategy profitability
    - Position size distribution shows risk taking
    - PnL by outcome shows which direction is more profitable
    """
    if df is None:
        print("No position data to plot")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Position Analysis', fontsize=14, fontweight='bold')
    
    # 1. Cash PnL distribution
    ax1 = axes[0, 0]
    pnl = df['cash_pnl'].dropna()
    ax1.hist(pnl, bins=50, edgecolor='black', alpha=0.7)
    ax1.axvline(x=0, color='red', linestyle='--', label='Break-even')
    ax1.axvline(x=pnl.mean(), color='green', linestyle='--', label=f'Mean: ${pnl.mean():.2f}')
    ax1.set_title('Cash PnL Distribution')
    ax1.set_xlabel('PnL ($)')
    ax1.set_ylabel('Frequency')
    ax1.legend()
    
    # 2. Percent PnL distribution
    ax2 = axes[0, 1]
    pct_pnl = df['percent_pnl'].dropna()
    ax2.hist(pct_pnl, bins=50, edgecolor='black', alpha=0.7)
    ax2.axvline(x=0, color='red', linestyle='--', label='Break-even')
    ax2.axvline(x=pct_pnl.mean(), color='green', linestyle='--', label=f'Mean: {pct_pnl.mean():.1f}%')
    ax2.set_title('Percent PnL Distribution')
    ax2.set_xlabel('PnL (%)')
    ax2.set_ylabel('Frequency')
    ax2.legend()
    
    # 3. Position size (shares) by outcome
    ax3 = axes[1, 0]
    if 'outcome' in df.columns:
        for outcome in df['outcome'].unique():
            subset = df[df['outcome'] == outcome]
            ax3.hist(subset['shares'], bins=30, alpha=0.5, label=outcome, edgecolor='black')
        ax3.set_title('Position Size by Outcome')
        ax3.set_xlabel('Shares')
        ax3.set_ylabel('Frequency')
        ax3.legend()
    
    # 4. PnL by outcome (box plot)
    ax4 = axes[1, 1]
    if 'outcome' in df.columns:
        outcomes = df['outcome'].unique()
        data_to_plot = [df[df['outcome'] == o]['cash_pnl'].values for o in outcomes]
        bp = ax4.boxplot(data_to_plot, labels=outcomes, patch_artist=True)
        colors = ['#3498db', '#e74c3c'][:len(outcomes)]
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        ax4.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        ax4.set_title('Cash PnL by Outcome')
        ax4.set_ylabel('PnL ($)')
    
    plt.tight_layout()
    plt.savefig(CONFIG['output_dir'] + 'position_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved: {CONFIG['output_dir']}position_analysis.png")


def plot_orderbook_analysis(df):
    """
    Create visualizations for orderbook data.
    
    JUSTIFICATION:
    - Spread distributions show trading costs
    - Implied probabilities show market pricing
    - Probability sum shows market efficiency
    """
    if df is None:
        print("No orderbook data to plot")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Orderbook Analysis', fontsize=14, fontweight='bold')
    
    # 1. UP spread distribution
    ax1 = axes[0, 0]
    ax1.hist(df['up_spread'], bins=50, edgecolor='black', alpha=0.7, label='UP')
    ax1.hist(df['down_spread'], bins=50, edgecolor='black', alpha=0.5, label='DOWN')
    ax1.set_title('Bid-Ask Spread Distribution')
    ax1.set_xlabel('Spread')
    ax1.set_ylabel('Frequency')
    ax1.legend()
    
    # 2. Implied probabilities over time
    ax2 = axes[0, 1]
    if 'up_midpoint' in df.columns and 'datetime' in df.columns:
        ax2.plot(df['datetime'], df['up_midpoint'], label='UP implied prob', alpha=0.7)
        ax2.plot(df['datetime'], df['down_midpoint'], label='DOWN implied prob', alpha=0.7)
        ax2.set_title('Implied Probabilities Over Time')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Probability')
        ax2.legend()
        ax2.tick_params(axis='x', rotation=45)
    
    # 3. Probability sum distribution (efficiency check)
    ax3 = axes[1, 0]
    if 'prob_sum_mid' in df.columns:
        ax3.hist(df['prob_sum_mid'], bins=50, edgecolor='black', alpha=0.7)
        ax3.axvline(x=1.0, color='red', linestyle='--', label='Efficient (1.0)')
        ax3.set_title('Probability Sum Distribution (UP + DOWN)')
        ax3.set_xlabel('Sum of Probabilities')
        ax3.set_ylabel('Frequency')
        ax3.legend()
    
    # 4. UP vs DOWN midpoint scatter
    ax4 = axes[1, 1]
    if 'up_midpoint' in df.columns:
        ax4.scatter(df['up_midpoint'], df['down_midpoint'], alpha=0.3, s=10)
        ax4.plot([0, 1], [1, 0], 'r--', label='Efficient frontier (sum=1)')
        ax4.set_title('UP vs DOWN Implied Probabilities')
        ax4.set_xlabel('UP Probability')
        ax4.set_ylabel('DOWN Probability')
        ax4.set_xlim(0, 1)
        ax4.set_ylim(0, 1)
        ax4.legend()
    
    plt.tight_layout()
    plt.savefig(CONFIG['output_dir'] + 'orderbook_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved: {CONFIG['output_dir']}orderbook_analysis.png")


def plot_orderbook_depth(df):
    """
    Additional orderbook analysis focusing on depth and liquidity.
    """
    if df is None:
        print("No orderbook data to plot")
        return
    
    if 'up_bid_depth_5' not in df.columns:
        print("No depth data available")
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle('Orderbook Depth Analysis', fontsize=14, fontweight='bold')
    
    # 1. Depth distribution
    ax1 = axes[0]
    ax1.hist(df['up_bid_depth_5'], bins=50, alpha=0.5, label='UP Bid Depth', edgecolor='black')
    ax1.hist(df['up_ask_depth_5'], bins=50, alpha=0.5, label='UP Ask Depth', edgecolor='black')
    ax1.set_title('UP Orderbook Depth Distribution')
    ax1.set_xlabel('Depth ($)')
    ax1.set_ylabel('Frequency')
    ax1.legend()
    
    # 2. Depth over time
    ax2 = axes[1]
    if 'datetime' in df.columns:
        ax2.plot(df['datetime'], df['up_bid_depth_5'], label='UP Bid', alpha=0.7)
        ax2.plot(df['datetime'], df['up_ask_depth_5'], label='UP Ask', alpha=0.7)
        ax2.set_title('UP Orderbook Depth Over Time')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Depth ($)')
        ax2.legend()
        ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(CONFIG['output_dir'] + 'orderbook_depth_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved: {CONFIG['output_dir']}orderbook_depth_analysis.png")


def main():
    """Generate all Phase 1 visualizations"""
    print("="*60)
    print(" PHASE 1 VISUALIZATIONS")
    print("="*60)
    print(f"\nOutput directory: {CONFIG['output_dir']}")
    
    # Load processed data
    print("\nLoading processed data...")
    
    try:
        btc_df = pd.read_parquet(CONFIG['btc_processed'])
        print(f"  ✓ BTC data: {len(btc_df):,} rows")
    except FileNotFoundError:
        print(f"  ✗ BTC data not found")
        btc_df = None
    
    try:
        trades_df = pd.read_parquet(CONFIG['trades_processed'])
        print(f"  ✓ Trades data: {len(trades_df):,} rows")
    except FileNotFoundError:
        print(f"  ✗ Trades data not found")
        trades_df = None
    
    try:
        positions_df = pd.read_parquet(CONFIG['positions_processed'])
        print(f"  ✓ Positions data: {len(positions_df):,} rows")
    except FileNotFoundError:
        print(f"  ✗ Positions data not found")
        positions_df = None
    
    try:
        orderbook_df = pd.read_parquet(CONFIG['orderbook_processed'])
        print(f"  ✓ Orderbook data: {len(orderbook_df):,} rows")
    except FileNotFoundError:
        print(f"  ✗ Orderbook data not found")
        orderbook_df = None
    
    # Generate visualizations
    print("\nGenerating visualizations...")
    
    plot_btc_analysis(btc_df)
    plot_trade_analysis(trades_df)
    plot_trade_price_analysis(trades_df)
    plot_position_analysis(positions_df)
    plot_orderbook_analysis(orderbook_df)
    plot_orderbook_depth(orderbook_df)
    
    print(f"\n✓ All visualizations saved to: {CONFIG['output_dir']}")
    print("\nGenerated files:")
    for f in os.listdir(CONFIG['output_dir']):
        if f.endswith('.png'):
            print(f"  - {f}")


if __name__ == "__main__":
    main()

 PHASE 1 VISUALIZATIONS

Output directory: analysis_output/charts/

Loading processed data...
  ✓ BTC data: 1,782 rows
  ✓ Trades data: 1,121 rows
  ✓ Positions data: 6,447 rows
  ✓ Orderbook data: 14,059 rows

Generating visualizations...
✓ Saved: analysis_output/charts/btc_analysis.png
✓ Saved: analysis_output/charts/trade_analysis.png
✓ Saved: analysis_output/charts/trade_price_analysis.png
✓ Saved: analysis_output/charts/position_analysis.png
✓ Saved: analysis_output/charts/orderbook_analysis.png
✓ Saved: analysis_output/charts/orderbook_depth_analysis.png

✓ All visualizations saved to: analysis_output/charts/

Generated files:
  - trade_price_analysis.png
  - position_analysis.png
  - orderbook_analysis.png
  - btc_analysis.png
  - orderbook_depth_analysis.png
  - trade_analysis.png


In [37]:
"""
================================================================================
PHASE 2: MERGED ANALYSIS & STRATEGY IDENTIFICATION
================================================================================
This script merges all 4 datasets to reconstruct the bot's decision-making
environment at each trade, then uses various techniques to identify the strategy.

JUSTIFICATION FOR PHASE 2:
Phase 1 showed us patterns in individual datasets. Now we need to understand
HOW these datasets relate to each other:
- What was BTC doing when the bot traded?
- What were the orderbook conditions?
- What was the bot's existing position?

By merging on timestamp, we reconstruct the EXACT market state at each decision.

Author: Strategy Analysis Pipeline
Date: December 2024
================================================================================
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For visualization
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_style("whitegrid")
    PLOTTING_AVAILABLE = True
except ImportError:
    PLOTTING_AVAILABLE = False

# For decision tree analysis
try:
    from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
    from sklearn.preprocessing import LabelEncoder
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False
    print("sklearn not available - decision tree analysis will be skipped")

import os

# ============================================================================
# CONFIGURATION
# ============================================================================

CONFIG = {
    # Input: processed files from Phase 1
    'btc_processed': 'analysis_output/btc_prices_processed.parquet',
    'trades_processed': 'analysis_output/trades_processed.parquet',
    'positions_processed': 'analysis_output/positions_processed.parquet',
    'orderbook_processed': 'analysis_output/orderbook_processed.parquet',
    
    # Output
    'output_dir': 'analysis_output/phase2/',
}

os.makedirs(CONFIG['output_dir'], exist_ok=True)


def print_section(title):
    print("\n" + "="*80)
    print(f" {title}")
    print("="*80)

def print_subsection(title):
    print(f"\n--- {title} ---")


# ============================================================================
# 2A. MERGE TRADES WITH BTC PRICES
# ============================================================================

def merge_trades_with_btc(trades_df, btc_df):
    """
    Merge trades with BTC prices to understand what triggered each trade.
    
    JUSTIFICATION:
    This is THE KEY ANALYSIS. By knowing the BTC price behavior at each trade,
    we can answer:
    - Does the bot trade after momentum (BTC going up → BUY UP)?
    - Does the bot trade mean-reversion (BTC going up → BUY DOWN)?
    - What time window of returns matters (1s? 5s? 30s? 60s)?
    
    Uses merge_asof with direction='backward' to get the MOST RECENT BTC price
    before each trade (the information the bot would have had).
    """
    print_section("2A. TRADES + BTC PRICES ANALYSIS")
    
    if trades_df is None or btc_df is None:
        print("Missing required data")
        return None
    
    # Ensure both are sorted by datetime
    trades_sorted = trades_df.sort_values('datetime').copy()
    btc_sorted = btc_df.sort_values('datetime').copy()
    
    print(f"Trades: {len(trades_sorted):,} rows")
    print(f"BTC prices: {len(btc_sorted):,} rows")
    
    # Merge: for each trade, get the most recent BTC price
    print("\nMerging on timestamp (backward asof merge)...")
    
    merged = pd.merge_asof(
        trades_sorted,
        btc_sorted[['datetime', 'price', 'return_1tick', 'return_5s', 'return_15s', 
                   'return_30s', 'return_60s', 'rolling_vol_60s']].rename(
            columns={
                'datetime': 'btc_datetime',
                'price': 'btc_price',
                'return_1tick': 'btc_return_1tick',
                'return_5s': 'btc_return_5s',
                'return_15s': 'btc_return_15s',
                'return_30s': 'btc_return_30s',
                'return_60s': 'btc_return_60s',
                'rolling_vol_60s': 'btc_vol_60s'
            }
        ),
        left_on='datetime',
        right_on='btc_datetime',
        direction='backward'
    )
    
    # Calculate time lag between BTC price and trade
    merged['btc_lag_seconds'] = (merged['datetime'] - merged['btc_datetime']).dt.total_seconds()
    
    print(f"\nMerged dataset: {len(merged):,} rows")
    print(f"Average BTC data lag: {merged['btc_lag_seconds'].mean():.2f} seconds")
    
    # ANALYSIS: What BTC returns precede each action?
    print_subsection("BTC Returns Before Each Action Type")
    
    for return_col in ['btc_return_5s', 'btc_return_15s', 'btc_return_30s', 'btc_return_60s']:
        print(f"\n{return_col}:")
        stats = merged.groupby('action')[return_col].agg(['mean', 'std', 'median'])
        stats['mean_pct'] = stats['mean'] * 100
        print(stats.round(6))
    
    # KEY INSIGHT: Direction correlation
    print_subsection("KEY INSIGHT: Does BTC direction predict trade direction?")
    
    # For each return window, check if positive returns → BUY UP (momentum)
    # or positive returns → BUY DOWN (mean reversion)
    
    for window in ['5s', '15s', '30s', '60s']:
        col = f'btc_return_{window}'
        
        # When BTC is going UP (positive return)
        btc_up = merged[merged[col] > 0]
        btc_down = merged[merged[col] < 0]
        
        if len(btc_up) > 0:
            buy_up_when_btc_up = (btc_up['action'] == 'BUY_Up').sum() / len(btc_up)
            buy_down_when_btc_up = (btc_up['action'] == 'BUY_Down').sum() / len(btc_up)
            
            print(f"\n{window} window:")
            print(f"  When BTC↑: BUY UP = {buy_up_when_btc_up*100:.1f}%, BUY DOWN = {buy_down_when_btc_up*100:.1f}%")
        
        if len(btc_down) > 0:
            buy_up_when_btc_down = (btc_down['action'] == 'BUY_Up').sum() / len(btc_down)
            buy_down_when_btc_down = (btc_down['action'] == 'BUY_Down').sum() / len(btc_down)
            
            print(f"  When BTC↓: BUY UP = {buy_up_when_btc_down*100:.1f}%, BUY DOWN = {buy_down_when_btc_down*100:.1f}%")
    
    # Save merged data
    output_path = CONFIG['output_dir'] + 'trades_with_btc.parquet'
    merged.to_parquet(output_path)
    print(f"\n✓ Saved: {output_path}")
    
    return merged


# ============================================================================
# 2B. MERGE TRADES WITH ORDERBOOK
# ============================================================================

def merge_trades_with_orderbook(trades_df, orderbook_df):
    """
    Merge trades with orderbook to understand execution context.
    
    JUSTIFICATION:
    The orderbook tells us:
    - What spread was the bot paying?
    - Was it taking liquidity or providing it?
    - What were the implied probabilities when it traded?
    - Was there any edge visible in the orderbook?
    """
    print_section("2B. TRADES + ORDERBOOK ANALYSIS")
    
    if trades_df is None or orderbook_df is None:
        print("Missing required data")
        return None
    
    trades_sorted = trades_df.sort_values('datetime').copy()
    orderbook_sorted = orderbook_df.sort_values('datetime').copy()
    
    print(f"Trades: {len(trades_sorted):,} rows")
    print(f"Orderbook snapshots: {len(orderbook_sorted):,} rows")
    
    # Merge
    print("\nMerging on timestamp...")
    
    merged = pd.merge_asof(
        trades_sorted,
        orderbook_sorted[[
            'datetime', 'up_best_bid', 'up_best_ask', 'up_spread',
            'down_best_bid', 'down_best_ask', 'down_spread',
            'up_midpoint', 'down_midpoint', 'prob_sum_mid'
        ]].rename(columns={'datetime': 'ob_datetime'}),
        left_on='datetime',
        right_on='ob_datetime',
        direction='backward'
    )
    
    merged['ob_lag_seconds'] = (merged['datetime'] - merged['ob_datetime']).dt.total_seconds()
    
    print(f"\nMerged dataset: {len(merged):,} rows")
    print(f"Average orderbook data lag: {merged['ob_lag_seconds'].mean():.2f} seconds")
    
    # ANALYSIS: Execution quality
    print_subsection("Execution Quality Analysis")
    
    # Compare trade price to best bid/ask
    # If buying: trade_price vs best_ask (should be <= ask)
    # If selling: trade_price vs best_bid (should be >= bid)
    
    for outcome in ['Up', 'Down']:
        print(f"\n{outcome} outcome trades:")
        outcome_trades = merged[merged['outcome'] == outcome]
        
        bid_col = f'{outcome.lower()}_best_bid'
        ask_col = f'{outcome.lower()}_best_ask'
        
        buys = outcome_trades[outcome_trades['side'] == 'BUY']
        sells = outcome_trades[outcome_trades['side'] == 'SELL']
        
        if len(buys) > 0:
            # For buys, compare trade price to ask
            buys['spread_paid'] = buys['price'] - buys[ask_col]
            print(f"  BUY trades: {len(buys)}")
            print(f"    Avg trade price: {buys['price'].mean():.4f}")
            print(f"    Avg best ask: {buys[ask_col].mean():.4f}")
            print(f"    Avg spread paid: {buys['spread_paid'].mean():.4f}")
        
        if len(sells) > 0:
            # For sells, compare trade price to bid
            sells['spread_received'] = sells[bid_col] - sells['price']
            print(f"  SELL trades: {len(sells)}")
            print(f"    Avg trade price: {sells['price'].mean():.4f}")
            print(f"    Avg best bid: {sells[bid_col].mean():.4f}")
            print(f"    Avg spread received: {sells['spread_received'].mean():.4f}")
    
    # ANALYSIS: What spreads exist when bot trades?
    print_subsection("Spread Conditions at Trade Time")
    
    print(f"UP spread at trade time:")
    print(f"  Mean: {merged['up_spread'].mean():.4f}")
    print(f"  Median: {merged['up_spread'].median():.4f}")
    
    print(f"\nDOWN spread at trade time:")
    print(f"  Mean: {merged['down_spread'].mean():.4f}")
    print(f"  Median: {merged['down_spread'].median():.4f}")
    
    # ANALYSIS: Implied probabilities when trading
    print_subsection("Implied Probabilities at Trade Time")
    
    print(f"UP implied probability when trading UP:")
    up_trades = merged[merged['outcome'] == 'Up']
    if len(up_trades) > 0:
        print(f"  Mean: {up_trades['up_midpoint'].mean():.4f} ({up_trades['up_midpoint'].mean()*100:.1f}%)")
    
    print(f"\nDOWN implied probability when trading DOWN:")
    down_trades = merged[merged['outcome'] == 'Down']
    if len(down_trades) > 0:
        print(f"  Mean: {down_trades['down_midpoint'].mean():.4f} ({down_trades['down_midpoint'].mean()*100:.1f}%)")
    
    # Save
    output_path = CONFIG['output_dir'] + 'trades_with_orderbook.parquet'
    merged.to_parquet(output_path)
    print(f"\n✓ Saved: {output_path}")
    
    return merged


# ============================================================================
# 2C. FULL STATE RECONSTRUCTION
# ============================================================================

def create_full_state_dataframe(trades_df, btc_df, orderbook_df, positions_df):
    """
    Create a comprehensive dataframe with EVERYTHING at each trade.
    
    JUSTIFICATION:
    This is the ultimate dataset for strategy identification.
    Each row = one trade, with columns for:
    - Trade details (side, price, size, outcome)
    - BTC state (price, returns at various windows, volatility)
    - Orderbook state (spreads, depths, implied probs)
    - Position state (current exposure, unrealized PnL)
    
    This lets us ask: "Given ALL this information, what did the bot do?"
    """
    print_section("2C. FULL STATE RECONSTRUCTION")
    
    if trades_df is None:
        print("No trades data - cannot proceed")
        return None
    
    # Start with trades
    df = trades_df.sort_values('datetime').copy()
    print(f"Starting with {len(df):,} trades")
    
    # Add BTC state
    if btc_df is not None:
        btc_sorted = btc_df.sort_values('datetime').copy()
        
        btc_cols = ['datetime', 'price']
        # Add return columns if they exist
        for col in btc_df.columns:
            if 'return' in col or 'vol' in col:
                btc_cols.append(col)
        
        btc_subset = btc_sorted[btc_cols].rename(columns={'datetime': 'btc_datetime', 'price': 'btc_price'})
        
        df = pd.merge_asof(
            df, btc_subset,
            left_on='datetime', right_on='btc_datetime',
            direction='backward'
        )
        print(f"  ✓ Added BTC state")
    
    # Add orderbook state
    if orderbook_df is not None:
        ob_sorted = orderbook_df.sort_values('datetime').copy()
        
        ob_cols = ['datetime', 'up_best_bid', 'up_best_ask', 'up_spread',
                  'down_best_bid', 'down_best_ask', 'down_spread']
        if 'up_midpoint' in ob_sorted.columns:
            ob_cols.extend(['up_midpoint', 'down_midpoint', 'prob_sum_mid'])
        if 'up_bid_depth_5' in ob_sorted.columns:
            ob_cols.extend(['up_bid_depth_5', 'up_ask_depth_5'])
        
        ob_subset = ob_sorted[ob_cols].rename(columns={'datetime': 'ob_datetime'})
        
        df = pd.merge_asof(
            df, ob_subset,
            left_on='datetime', right_on='ob_datetime',
            direction='backward'
        )
        print(f"  ✓ Added orderbook state")
    
    # Add position state (this is trickier - need most recent position snapshot)
    if positions_df is not None:
        # Positions might not have a simple datetime - need to handle this
        # For now, we'll skip if timestamps don't align well
        print(f"  ⚠ Position merge may need custom handling based on your data format")
    
    # Feature engineering
    print_subsection("Feature Engineering")
    
    # BTC momentum indicators
    if 'btc_return_30s' in df.columns:
        df['btc_momentum'] = np.sign(df['btc_return_30s'])
        print("  ✓ Added btc_momentum (sign of 30s return)")
    
    # Orderbook features
    if 'up_spread' in df.columns and 'down_spread' in df.columns:
        df['total_spread'] = df['up_spread'] + df['down_spread']
        df['spread_ratio'] = df['up_spread'] / df['down_spread'].replace(0, np.nan)
        print("  ✓ Added spread features")
    
    if 'up_midpoint' in df.columns and 'down_midpoint' in df.columns:
        # Deviation from 50/50
        df['up_implied_deviation'] = df['up_midpoint'] - 0.5
        df['market_inefficiency'] = abs(df['prob_sum_mid'] - 1.0)
        print("  ✓ Added probability features")
    
    # Time features
    if 'seconds_into_window' in df.columns:
        df['window_pct'] = df['seconds_into_window'] / 900  # 900 seconds = 15 min
        print("  ✓ Added window timing features")
    
    # Create binary target for prediction
    df['is_buy'] = (df['side'] == 'BUY').astype(int)
    df['is_up'] = (df['outcome'] == 'Up').astype(int)
    
    print(f"\nFinal dataset: {len(df):,} rows, {len(df.columns)} columns")
    print(f"\nColumns: {list(df.columns)}")
    
    # Save
    output_path = CONFIG['output_dir'] + 'full_state_reconstruction.parquet'
    df.to_parquet(output_path)
    print(f"\n✓ Saved: {output_path}")
    
    return df


# ============================================================================
# 2D. STRATEGY IDENTIFICATION
# ============================================================================

def identify_strategy_with_decision_tree(df):
    """
    Use decision trees to extract interpretable rules.
    
    JUSTIFICATION:
    Decision trees are perfect for reverse-engineering because:
    - They produce human-readable rules
    - They automatically find important thresholds
    - They reveal which features matter most
    
    We train separate trees for:
    1. Predicting BUY vs SELL
    2. Predicting UP vs DOWN
    3. Predicting the full action (BUY_UP, SELL_UP, etc.)
    """
    print_section("2D. STRATEGY IDENTIFICATION (Decision Trees)")
    
    if not SKLEARN_AVAILABLE:
        print("sklearn not available - skipping decision tree analysis")
        return None
    
    if df is None:
        print("No data available")
        return None
    
    # Select features for the model
    potential_features = [
        # BTC features
        'btc_return_5s', 'btc_return_15s', 'btc_return_30s', 'btc_return_60s',
        'btc_momentum',
        # Orderbook features
        'up_spread', 'down_spread', 'total_spread', 'spread_ratio',
        'up_midpoint', 'down_midpoint', 'up_implied_deviation', 'market_inefficiency',
        # Timing features
        'seconds_into_window', 'window_pct'
    ]
    
    # Filter to features that exist
    features = [f for f in potential_features if f in df.columns]
    print(f"Using features: {features}")
    
    if len(features) < 2:
        print("Not enough features for decision tree analysis")
        return None
    
    # Prepare data
    X = df[features].copy()
    
    # Handle missing values
    X = X.fillna(X.median())
    
    # Drop rows where we still have issues
    valid_mask = ~X.isnull().any(axis=1)
    X = X[valid_mask]
    df_valid = df[valid_mask]
    
    print(f"\nValid samples for modeling: {len(X):,}")
    
    # =====================
    # MODEL 1: Predict BUY vs SELL
    # =====================
    print_subsection("Model 1: Predicting BUY vs SELL")
    
    y_side = df_valid['is_buy']
    
    tree_side = DecisionTreeClassifier(max_depth=4, min_samples_leaf=10, random_state=42)
    tree_side.fit(X, y_side)
    
    print(f"Accuracy: {tree_side.score(X, y_side):.2%}")
    print(f"\nFeature Importances:")
    for feat, imp in sorted(zip(features, tree_side.feature_importances_), key=lambda x: -x[1]):
        if imp > 0.01:
            print(f"  {feat}: {imp:.3f}")
    
    print(f"\nDecision Rules:")
    print(export_text(tree_side, feature_names=features, max_depth=3))
    
    # =====================
    # MODEL 2: Predict UP vs DOWN outcome
    # =====================
    print_subsection("Model 2: Predicting UP vs DOWN Outcome")
    
    y_outcome = df_valid['is_up']
    
    tree_outcome = DecisionTreeClassifier(max_depth=4, min_samples_leaf=10, random_state=42)
    tree_outcome.fit(X, y_outcome)
    
    print(f"Accuracy: {tree_outcome.score(X, y_outcome):.2%}")
    print(f"\nFeature Importances:")
    for feat, imp in sorted(zip(features, tree_outcome.feature_importances_), key=lambda x: -x[1]):
        if imp > 0.01:
            print(f"  {feat}: {imp:.3f}")
    
    print(f"\nDecision Rules:")
    print(export_text(tree_outcome, feature_names=features, max_depth=3))
    
    # =====================
    # MODEL 3: Predict Full Action
    # =====================
    print_subsection("Model 3: Predicting Full Action")
    
    if 'action' in df_valid.columns:
        le = LabelEncoder()
        y_action = le.fit_transform(df_valid['action'])
        
        tree_action = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, random_state=42)
        tree_action.fit(X, y_action)
        
        print(f"Accuracy: {tree_action.score(X, y_action):.2%}")
        print(f"Classes: {le.classes_}")
        print(f"\nFeature Importances:")
        for feat, imp in sorted(zip(features, tree_action.feature_importances_), key=lambda x: -x[1]):
            if imp > 0.01:
                print(f"  {feat}: {imp:.3f}")
        
        print(f"\nDecision Rules:")
        print(export_text(tree_action, feature_names=features, 
                         class_names=list(le.classes_), max_depth=3))
    
    # Save visualization if matplotlib available
    if PLOTTING_AVAILABLE:
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))
        
        plot_tree(tree_side, feature_names=features, class_names=['SELL', 'BUY'],
                 filled=True, ax=axes[0], max_depth=3, fontsize=8)
        axes[0].set_title('Decision Tree: BUY vs SELL')
        
        plot_tree(tree_outcome, feature_names=features, class_names=['DOWN', 'UP'],
                 filled=True, ax=axes[1], max_depth=3, fontsize=8)
        axes[1].set_title('Decision Tree: UP vs DOWN Outcome')
        
        plt.tight_layout()
        plt.savefig(CONFIG['output_dir'] + 'decision_trees.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"\n✓ Saved decision tree visualization")
    
    return {
        'tree_side': tree_side,
        'tree_outcome': tree_outcome,
        'features': features
    }


# ============================================================================
# 2E. THRESHOLD DETECTION
# ============================================================================

def detect_thresholds(df):
    """
    Find specific thresholds that trigger trading decisions.
    
    JUSTIFICATION:
    Even if we don't use ML, we can find thresholds by:
    - Comparing distributions of features split by action
    - Looking for clear separation points
    - Testing specific threshold hypotheses
    """
    print_section("2E. THRESHOLD DETECTION")
    
    if df is None:
        print("No data available")
        return
    
    # For each numeric feature, compare distributions by action
    print_subsection("Feature Distributions by Action")
    
    numeric_features = ['btc_return_5s', 'btc_return_15s', 'btc_return_30s', 'btc_return_60s',
                       'up_spread', 'down_spread', 'seconds_into_window']
    
    for feat in numeric_features:
        if feat not in df.columns:
            continue
        
        print(f"\n{feat}:")
        stats = df.groupby('action')[feat].agg(['mean', 'median', 'std'])
        print(stats.round(6))
    
    # Specific hypothesis tests
    print_subsection("Hypothesis Tests")
    
    # Hypothesis 1: Bot buys UP when BTC is rising
    if 'btc_return_30s' in df.columns:
        print("\nH1: Bot buys UP when BTC 30s return > 0")
        
        buy_up = df[df['action'] == 'BUY_Up']
        buy_down = df[df['action'] == 'BUY_Down']
        
        if len(buy_up) > 0 and len(buy_down) > 0:
            pct_buy_up_when_btc_positive = (buy_up['btc_return_30s'] > 0).mean() * 100
            pct_buy_down_when_btc_positive = (buy_down['btc_return_30s'] > 0).mean() * 100
            
            print(f"  BUY UP trades with positive BTC return: {pct_buy_up_when_btc_positive:.1f}%")
            print(f"  BUY DOWN trades with positive BTC return: {pct_buy_down_when_btc_positive:.1f}%")
            
            if pct_buy_up_when_btc_positive > 60:
                print("  → MOMENTUM STRATEGY: Buying direction of BTC movement")
            elif pct_buy_up_when_btc_positive < 40:
                print("  → MEAN REVERSION STRATEGY: Fading BTC movement")
            else:
                print("  → MIXED or COMPLEX strategy")
    
    # Hypothesis 2: Bot trades more actively early in window
    if 'seconds_into_window' in df.columns:
        print("\nH2: Bot trades early in 15-min window")
        
        early = (df['seconds_into_window'] < 300).mean() * 100  # First 5 min
        mid = ((df['seconds_into_window'] >= 300) & (df['seconds_into_window'] < 600)).mean() * 100
        late = (df['seconds_into_window'] >= 600).mean() * 100
        
        print(f"  Early (0-5 min): {early:.1f}%")
        print(f"  Mid (5-10 min): {mid:.1f}%")
        print(f"  Late (10-15 min): {late:.1f}%")
    
    # Hypothesis 3: Bot only trades when spread is below threshold
    if 'up_spread' in df.columns:
        print("\nH3: Bot trades only when spread < threshold")
        
        spread_percentiles = df['up_spread'].quantile([0.25, 0.5, 0.75])
        print(f"  UP spread percentiles: 25%={spread_percentiles[0.25]:.4f}, " +
              f"50%={spread_percentiles[0.5]:.4f}, 75%={spread_percentiles[0.75]:.4f}")


# ============================================================================
# 2F. EVENT STUDY
# ============================================================================

def event_study(trades_df, btc_df, window_before=60, window_after=60):
    """
    Align all trades to t=0 and plot average BTC price around trades.
    
    JUSTIFICATION:
    This visually reveals what BTC was doing before and after trades.
    If bot uses momentum: BTC will be moving in trade direction before entry
    If bot uses mean-reversion: BTC will be moving opposite before entry
    """
    print_section("2F. EVENT STUDY (Price Action Around Trades)")
    
    if trades_df is None or btc_df is None:
        print("Missing required data")
        return
    
    if not PLOTTING_AVAILABLE:
        print("matplotlib not available - skipping event study visualization")
        return
    
    print(f"Analyzing price action from -{window_before}s to +{window_after}s around trades")
    
    btc_sorted = btc_df.sort_values('datetime')
    
    # For each action type, collect price paths
    action_paths = {}
    
    for action in trades_df['action'].unique():
        action_trades = trades_df[trades_df['action'] == action]
        paths = []
        
        for _, trade in action_trades.iterrows():
            trade_time = trade['datetime']
            start_time = trade_time - timedelta(seconds=window_before)
            end_time = trade_time + timedelta(seconds=window_after)
            
            # Get BTC prices in this window
            window_btc = btc_sorted[
                (btc_sorted['datetime'] >= start_time) & 
                (btc_sorted['datetime'] <= end_time)
            ].copy()
            
            if len(window_btc) > 10:  # Need enough data points
                # Normalize: set t=0 price to 0 (show relative change)
                window_btc['rel_time'] = (window_btc['datetime'] - trade_time).dt.total_seconds()
                
                # Get price at t=0 (closest to trade time)
                t0_idx = (window_btc['rel_time'].abs()).idxmin()
                t0_price = window_btc.loc[t0_idx, 'price']
                
                # Express as percentage change from t=0
                window_btc['rel_price'] = (window_btc['price'] / t0_price - 1) * 100
                
                paths.append(window_btc[['rel_time', 'rel_price']])
        
        if paths:
            action_paths[action] = paths
            print(f"  {action}: {len(paths)} valid trade windows")
    
    # Plot average paths
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = {'BUY_Up': 'green', 'SELL_Up': 'lightgreen', 
              'BUY_Down': 'red', 'SELL_Down': 'lightcoral'}
    
    for action, paths in action_paths.items():
        # Interpolate all paths to common time grid
        time_grid = np.linspace(-window_before, window_after, 121)  # 1-second resolution
        
        interp_paths = []
        for path_df in paths:
            interp_price = np.interp(time_grid, path_df['rel_time'], path_df['rel_price'])
            interp_paths.append(interp_price)
        
        # Calculate mean and confidence interval
        interp_paths = np.array(interp_paths)
        mean_path = np.mean(interp_paths, axis=0)
        std_path = np.std(interp_paths, axis=0)
        
        color = colors.get(action, 'gray')
        ax.plot(time_grid, mean_path, label=f'{action} (n={len(paths)})', 
                color=color, linewidth=2)
        ax.fill_between(time_grid, mean_path - std_path, mean_path + std_path,
                       alpha=0.2, color=color)
    
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.5, label='Trade Time')
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    ax.set_xlabel('Seconds Relative to Trade')
    ax.set_ylabel('BTC Price Change (%)')
    ax.set_title('Event Study: BTC Price Action Around Bot Trades')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(CONFIG['output_dir'] + 'event_study.png', dpi=150, bbox_inches='tight')
    plt.close()
    print(f"\n✓ Saved: {CONFIG['output_dir']}event_study.png")


# ============================================================================
# SUMMARY REPORT
# ============================================================================

def generate_phase2_summary(df):
    """Generate a summary of Phase 2 findings"""
    print_section("PHASE 2 SUMMARY: STRATEGY HYPOTHESIS")
    
    if df is None:
        print("No data available for summary")
        return
    
    print("\n📊 STRATEGY INDICATORS")
    print("-" * 40)
    
    # Direction logic
    if 'btc_return_30s' in df.columns:
        buy_up = df[df['action'] == 'BUY_Up']
        buy_down = df[df['action'] == 'BUY_Down']
        
        if len(buy_up) > 0:
            btc_up_before_buy_up = (buy_up['btc_return_30s'] > 0).mean()
            print(f"\n1. DIRECTION LOGIC:")
            print(f"   BTC rising (30s) before BUY UP: {btc_up_before_buy_up*100:.1f}%")
            
            if btc_up_before_buy_up > 0.6:
                print(f"   → MOMENTUM: Bot follows BTC direction")
            elif btc_up_before_buy_up < 0.4:
                print(f"   → MEAN REVERSION: Bot fades BTC moves")
            else:
                print(f"   → COMPLEX: Not simple momentum or reversion")
    
    # Timing logic
    if 'seconds_into_window' in df.columns:
        avg_entry_time = df['seconds_into_window'].mean()
        print(f"\n2. TIMING LOGIC:")
        print(f"   Average entry: {avg_entry_time:.0f}s into window")
        print(f"   ({avg_entry_time/60:.1f} minutes)")
    
    # Sizing logic
    size_std = df['size'].std()
    size_mean = df['size'].mean()
    cv = size_std / size_mean if size_mean > 0 else 0
    print(f"\n3. SIZING LOGIC:")
    print(f"   Size CV (std/mean): {cv:.2f}")
    if cv < 0.1:
        print(f"   → FIXED SIZING: Nearly constant position size")
    else:
        print(f"   → VARIABLE SIZING: Adaptive position sizing")
    
    # Overall summary
    print(f"\n" + "="*40)
    print("PRELIMINARY STRATEGY DESCRIPTION:")
    print("="*40)
    print("\n[Fill this in based on the analysis results above]")
    print("\nThe bot appears to:")
    print("- Entry trigger: [momentum / mean-reversion / spread-based]")
    print("- Exit trigger: [time-based / profit-target / price-based]")
    print("- Position sizing: [fixed / scaled]")
    print("- Timing: [early / mid / late window]")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Run the complete Phase 2 analysis"""
    print("="*80)
    print(" POLYMARKET BOT STRATEGY ANALYSIS - PHASE 2")
    print(" Merged Analysis & Strategy Identification")
    print("="*80)
    print(f"\nStarted at: {datetime.now()}")
    
    # Load processed data from Phase 1
    print("\nLoading Phase 1 processed data...")
    
    try:
        btc_df = pd.read_parquet(CONFIG['btc_processed'])
        print(f"  ✓ BTC: {len(btc_df):,} rows")
    except FileNotFoundError:
        print(f"  ✗ BTC data not found - run Phase 1 first")
        btc_df = None
    
    try:
        trades_df = pd.read_parquet(CONFIG['trades_processed'])
        print(f"  ✓ Trades: {len(trades_df):,} rows")
    except FileNotFoundError:
        print(f"  ✗ Trades data not found - run Phase 1 first")
        trades_df = None
    
    try:
        positions_df = pd.read_parquet(CONFIG['positions_processed'])
        print(f"  ✓ Positions: {len(positions_df):,} rows")
    except FileNotFoundError:
        print(f"  ✗ Positions data not found")
        positions_df = None
    
    try:
        orderbook_df = pd.read_parquet(CONFIG['orderbook_processed'])
        print(f"  ✓ Orderbook: {len(orderbook_df):,} rows")
    except FileNotFoundError:
        print(f"  ✗ Orderbook data not found")
        orderbook_df = None
    
    if trades_df is None:
        print("\n✗ Cannot proceed without trades data")
        return
    
    # Run analyses
    trades_btc = merge_trades_with_btc(trades_df, btc_df)
    trades_ob = merge_trades_with_orderbook(trades_df, orderbook_df)
    
    full_df = create_full_state_dataframe(trades_df, btc_df, orderbook_df, positions_df)
    
    models = identify_strategy_with_decision_tree(full_df)
    detect_thresholds(full_df)
    event_study(trades_df, btc_df)
    
    generate_phase2_summary(full_df)
    
    print(f"\n✓ Phase 2 analysis complete at: {datetime.now()}")
    print(f"✓ Output saved to: {CONFIG['output_dir']}")


if __name__ == "__main__":
    main()

 POLYMARKET BOT STRATEGY ANALYSIS - PHASE 2
 Merged Analysis & Strategy Identification

Started at: 2025-12-22 22:06:55.961763

Loading Phase 1 processed data...
  ✓ BTC: 1,782 rows
  ✓ Trades: 1,121 rows
  ✓ Positions: 6,447 rows
  ✓ Orderbook: 14,059 rows

 2A. TRADES + BTC PRICES ANALYSIS
Trades: 1,121 rows
BTC prices: 1,782 rows

Merging on timestamp (backward asof merge)...

Merged dataset: 1,121 rows
Average BTC data lag: 0.60 seconds

--- BTC Returns Before Each Action Type ---

btc_return_5s:
              mean       std    median  mean_pct
action                                          
BUY_Down -0.000083  0.000320 -0.000064 -0.008296
BUY_Up    0.000049  0.000282  0.000030  0.004856

btc_return_15s:
              mean       std    median  mean_pct
action                                          
BUY_Down -0.000134  0.000563 -0.000065 -0.013418
BUY_Up   -0.000044  0.000537 -0.000019 -0.004392

btc_return_30s:
              mean       std    median  mean_pct
action             